# Aggregation with groupby

In [ ]:
import pandas as pd
import numpy as np

# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Set to 2 decimal places, and display all columns
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Load the survey data

In [ ]:
# Read 'cleaned_survey.csv' file
df  = pd.read_csv('cleaned_survey.csv', index_col=0)

In [ ]:
df.head(10)

## groupby

The method <i>groupby</i> splits the data by the value of a field <i>f</i>. We can then aggregate other columns separately for each value of <i>f</i>

<b>Example</b>: We know how to show the average value of each column.  But groupby allows us to show the average value of each column divided by "Program"

In [ ]:
# Show mean of all columns
df.???()

In [ ]:
# Show the average value of each column grouping by "Program"
df.???('Program').???()

#### Example

Display the mean of all columns, grouping by the Job situation

In [ ]:
# Show the average value of each column grouping by "Job"
df.???('Job').???()

## .groupby() object

.groupby() generates how many groups? 

In [ ]:
# Check the type
???(df.groupby('Job'))

In [ ]:
# Check the number of group (ngroups)
df.groupby('Job').???

In [ ]:
# List all groups (groups)
df.groupby('Job').???

## Aggregate only some columns

Oftentimes, we don't want to aggregate all columns. For example, we want to find the average of Job grouped by Program.

In [ ]:
# Show the average of Job grouped by Program
df.groupby('Program')[???].mean()

Or more columns. For example, we want to find the average of Job, C, and R, grouped by Program.

In [ ]:
# Show the average of Job, C, and R, grouped by Program.
# You need to put all columns' index in a list ['Job','C','R']
df.groupby('Program')[???].mean()

## Problems

For each Job situation (0=no job, 0.5=part time, 1=full time), find the proportion of students that know SQL

In [ ]:
# We can use mean to get the proportion since SQL column has value 1 or 0
df.groupby('Job')['SQL'].???()

For each program, count how many student know SQL.

use count() ?  use count() will count in both 0 and 1

In [ ]:
# We cannot use count since will count in both 0 and 1. We only need the number of value 1 here.
df.groupby('Program')['SQL'].count()

so should use sum() instead

In [ ]:
# We need to use sum to get the number of value 1.
df.groupby('Program')['SQL'].???()

Considering only the students who know SQL, find for each Program the proportion of students who know Java

In [ ]:
# Get the subset of students who know SQL first students who know SQL first (df.SQL==1)
df[???].groupby('Program')['Java'].mean()

Next two commands will lead to same answer, but which one is faster? Why?
<ol>
<li>df.groupby(by='Program')['SQL'].mean()
<li>df.groupby(by='Program').mean()['SQL']
</ol>

In [ ]:
%timeit df.groupby(by='Program')['SQL'].mean()

In [ ]:
%timeit df.groupby(by='Program').mean()['SQL']

For each Classification skill level, how many MBA students are there? Your result should have 5 rows (one for each classification skill level: 1, 2, 3, 4, and 5)

Here is the wrong way to do it:
<ol>
<li>Keep only the rows of MBA students
<li>Perform group by
</ol>

In [ ]:
# There is no skill level 5 output.
df[df.Program=='MBA'].groupby('Classification')['Classification'].size()

<p>Here is the correct way to do it:</p>
<ol>
<li>Create a dummy variable 'MBA' that has a 1 (or True) if the student is an MBA student and 0 (or False) otherwise
<li>For each classification level, compute the sum of the 'MBA' column. Note that the sum of boolean values counts the True values.
<li>Remove the dummy variable 'MBA'
</ol>

In [ ]:
df.head()

In [ ]:
# Create a dummy variable 'MBA' that has a 1 (or True) if the student is an MBA student and 0 (or False) otherwise
# Covert boolean to 1/0
df['MBA'] = (df.Program == 'MBA') +???

In [ ]:
# For each classification level, compute the sum of the 'MBA' column. 
df.groupby('Classification')['MBA'].???()

## Apply multiple functions (<i>agg</i>)

For each Job situation (0=no job, 0.5=part time, 1=full time), find (1) how many students are in that Job situation and (2) the proportion of students that know SQL.

Let's check the NaN in the DataFrame first ..

In [ ]:
# You can use isnull() to check whether the value is missing
df.???()

In [ ]:
# You can use isna() to check whether the value is missing
df.???()

In [ ]:
# The any() method returns one value for each column, True if ANY value in that column is True, otherwise False.
df.isna().???()

In [ ]:
# Apply any() function again will tell you whether there are any missing value in the dataset
df.isna().any().???()

In [ ]:
# use sum() after isna() can give you the number of missing value in each column
df.isna().???()

In [ ]:
# Apply sum() function again will tell the number of missing value in the dataframe
df.isna().sum().???()

Now we know which columns has Nan

'Job' column doesn't have Nan, so .count() and .size() return the same. so 'how many students are in that Job situation' can use either one. 

In [ ]:
# apply count() to SQL for each Job group
df.groupby('Job')['SQL'].???()

In [ ]:
# apply size() to SQL for each Job group
df.groupby('Job')['SQL'].???()

In [ ]:
# apply mean() to SQL for each Job group
df.groupby('Job')['SQL'].???()

But column 'SQL' has Nan, so when couting the 'Job' situation through the 'SQL' column, we need to use *size* instead of *count*

In [ ]:
df.groupby('Job')['SQL'].agg(['mean',???])

In [ ]:
# with .count() will lead to incorrect answer.
df.groupby('Job')['SQL'].agg(['mean',???])

#### Renaming resulting columns

You need to rename the columns manually

In [ ]:
# Rename 'mean'to'SQL_porpotion', 'size' to 'N_students'
df.groupby('Job')['SQL'].agg(['mean','size']).???(columns={'mean':???, 'size':???})

## Apply multiple arbitrary functions to multiple columns and give them names (agg)

For each Job situation (0=no job, 0.5=part time, 1=full time), compute the average knowledge of SQL, the maximum knowledge of Classification, and the gap between the max and the min Classification score for each Job level

In [ ]:
# to apply functions to several columns, you need to put them in a dictionary structure ( {} and key value pair)
# Gap : lambda x: x.max()-x.min()
df.groupby('Job').agg({'SQL':'mean','Classification':['max', ???]})

We can also give a name to all columns created

In [ ]:
df.groupby('Job').agg({'SQL':'mean','Classification':['max', lambda x: x.max()-x.min()]}).\
    ???(columns={'mean':'SQL_avg','max':'max_Class','<lambda_0>':'spread_Class'})

## group by multiple fields

You can also group by multiple fields. For example, find the mean of all columns grouped by Program and Job situation.

In [ ]:
# grouped by Program and Job situation.
df.groupby([???,???]).mean()

It returns a DataFrame with a <b>Hierarchical Index</b> (i.e., a composite key in a database). In this case, the index is (Program,Job). DataFrames with Hierarchical Indeces are outside the scope of this course because they tend to be hard to deal with; you can avoid them here by using <i>as_index = False</i> inside the <i>groupby</i>.

In [ ]:
# Avoid Hierarchical Index by using as_index = False inside the groupby
df.groupby(['Program','Job'], as_index=False).mean()

Or use .reset_index()

In [ ]:
res = df.groupby(['Program','Job']).mean()

In [ ]:
# Avoid Hierarchical Index by using reset_index()
res.reset_index()

## Problems

Find the maximum, minimum, and average number of Languages known by students in each Program

In [ ]:
df.groupby('Program').agg({???:['max','min','mean']})

In [ ]:
# Apply function to a single column can be faster
df.groupby('Program')[???].agg(['max','min','mean'])

For each existing combination of programming skills level and Program, report the number of students (call it <i>nStudents</i>) and the proportion that know Python (call it <i>PythonProportion</i>)

In [ ]:
# check nan first isna()
df.Python.???().any()

In [ ]:
# The question is asking for combination of programming skills level and Program
# group by multiple columns ['ProgSkills','Program']
df.groupby(['ProgSkills','Program']).agg({'Python':['mean','size']})

In [ ]:
# rename the column index, 'mean' to 'PythonProportion','size' to 'nStudents'
df.groupby(['ProgSkills','Program']).agg({'Python':['mean','size']}).\
    ???(columns={'mean':'PythonProportion','size':'nStudents'})

### How to flatten a hierarchical index?

In [ ]:
# use as_index inside or use reset_index() to avoid hierarchical index
df.groupby(['ProgSkills','Program'], ???=False).agg({'Python':['mean','size']}).\
    rename(columns={'mean':'PythonProportion','size':'nStudents'})

HARD. For each Program, report:
<ul>
<li>the number of students who know both Python and C (call it <i>C_Python_Students</i>, and note that it can be equal to 0)
    <li>the gap between <b>max</b> and <b>mean</b> Clustering knowledge (call it <i>CluGap</i>)
</ul>

In [ ]:
# Generate a boolean series with two condition df.C==1.0 and df.Python==1.0
df['C_Python'] = (???) +0.0

In [ ]:
df.head()

In [ ]:
# Use sum to get the number of students who know both Python and C
# Write a lambda function to get the gap between max and mean Clustering knowledge
# lambda x: x.max()-x.mean()
df.groupby('Program').agg({'C_Python':???,'Clustering': ???})

In [ ]:
# Rename the columns
df.groupby('Program').agg({'C_Python':'sum','Clustering': lambda x: x.max()-x.mean()})\
   .???(columns={'Clustering':'CluGap','C_Python':'C_Python_Students'})

## Advanced: retrieve unaggregated rows (<i>apply</i>)

Sometimes, for each group-by value we want to retrieve one or more rows. For example, for each program report the student who knows most languages (report more than one students in case of ties)

In [ ]:
df_MBA= df[df.Program=='MBA']

In [ ]:
df_MBA[df_MBA.Languages == df_MBA.Languages.max()]

With *.groupby('Program')* ...

In [ ]:
df.groupby('Program').groups

In [ ]:
# Apply a lambda function to each group
df.groupby('Program').apply(lambda d: d[d.Languages == d.Languages.max()])

### Warning! Do not use DataFrameGroupBy.apply unless you actually need it

The method <i>DataFrameGroupBy.apply</i> is slow. It is implemented as a for loop that invokes the lambda function at each iteration.  

The fast way to compute the average ProgSkills for each program:

In [ ]:
%timeit df.groupby('Program')['ProgSkills'].mean()

The <b>slow</b> way to compute the average ProgSkills for each program:

In [ ]:
%timeit df.groupby('Program').apply(lambda d: d.ProgSkills.mean())

## Problems

For each ProgSkills level, find the student (or students in case of ties) with the highest Classification skills and show their knowledge of C and Java

In [ ]:
df[df.Classification==df.Classification.max()]

In [ ]:
df.groupby('ProgSkills').apply(lambda d: ???)

In [ ]:
# report 'Classification','C','Java' reults only
df.groupby('ProgSkills').apply(lambda d: d[d.Classification==d.Classification.max()])[['Classification','C','Java']]

For each ProgSkills level, find the Program with most students that have that ProgSkill level

Starts from ProgSkills level == 1 ..

In [ ]:
# get the df that df.ProgSkills==1
ProgSkill_level_1 = df[???]

In [ ]:
ProgSkill_level_1.groupby('Program').size().nlargest(1)

In [ ]:
df.groupby('ProgSkills').apply(lambda d:???)